In [1]:
# VGG16 모델 미세 튜닝
# 망을 동결하고, 1000개의 라벨을 가진 VGG16 의 마지막 레이어를 제거
import numpy as np
import keras
from keras.layers import Dense

vgg_model = keras.applications.vgg16.VGG16()
vgg_model.summary()

Using TensorFlow backend.












Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)   

In [2]:
last_layer = str(vgg_model.layers[-1])
last_layer

'<keras.layers.core.Dense object at 0x7f899c870908>'

In [3]:
classifier = keras.Sequential()
for layer in vgg_model.layers:
    if str(layer) != last_layer:
        classifier.add(layer)

In [4]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [5]:
# 레이어 동결
for layer in classifier.layers:
    layer.trainable=False

In [6]:
# 새로운 레이어를 추가하고, 요약 정보를 확인
classifier.add(Dense(1, activation='sigmoid'))
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [7]:
# 망을 컴파일
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [9]:
# 이미지를 처리하고, ANN 피팅 진행
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('Data/Dataset/training_set', target_size=(224,224), batch_size=32, class_mode='binary')
test_set = test_datagen.flow_from_directory('Data/Dataset/test_set', target_size=(224,224), batch_size=32, class_mode='binary')

classifier.fit_generator(training_set, steps_per_epoch=100, epochs=10, validation_data=test_set, validation_steps=30)

Found 200 images belonging to 2 classes.
Found 115 images belonging to 2 classes.


Epoch 1/10
100/100 [==============================] - 23s 230ms/step - loss: 0.5563 - acc: 0.7132 - val_loss: 0.3538 - val_acc: 0.8539
Epoch 2/10
100/100 [==============================] - 20s 202ms/step - loss: 0.3881 - acc: 0.8562 - val_loss: 0.3359 - val_acc: 0.8680
Epoch 3/10
100/100 [==============================] - 21s 205ms/step - loss: 0.3337 - acc: 0.8672 - val_loss: 0.3071 - val_acc: 0.8677
Epoch 4/10
100/100 [==============================] - 21s 207ms/step - loss: 0.2904 - acc: 0.8850 - val_loss: 0.2715 - val_acc: 0.8890
Epoch 5/10
100/100 [==============================] - 20s 201ms/step - loss: 0.2809 - acc: 0.8856 - val_loss: 0.2865 - val_acc: 0.8688
Epoch 6/10
100/100 [==============================] - 20s 203ms/step - loss: 0.2370 - acc: 0.9191 - val_loss: 0.2729 - val_acc: 0.8773
Epoch 7/10
100/100 [==============================] - 21s 207ms/step - loss: 0.2371 - acc: 0.9185 - val_lo

In [ ]:
from keras.preprocessing import image
import numpy as np

new_image = image.load_img('Data/Prediction/test_image_1.jpg', target_size=(224,224))
new_image = image.img_to_array(new_image)
new_image = np.expand_dims(new_image, axis=0)

result = classifier.predict(new_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'It is a Dog'
else:
    prediction = 'It is a Cat'
print(prediction)